In [ ]:
# Estos installs son innecesarios si usan el venv

# !pip install -U langchain_community --quiet
# !pip install pypdf --quiet
# !pip install instructor --quiet
# !pip install faiss-cpu

In [ ]:
# !pip install "numpy<2" #### no sé por qué estaba esta pero no es necesario para que corra este notebook y no pinta pq conflicta con otras cosas en el env hacer esto, varios modulos piden numpy > 2, es peligrosa.
# !pip install torch

In [3]:
import os, json, glob
import torch
import requests
import numpy as np
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from embeddings_wrapper import LangchainSentenceTransformer
from typing import Any
from pydantic import BaseModel, ValidationError, Field
from openai import AsyncAzureOpenAI


/Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/nlp_venv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Seleccionar parámetros en la siguiente celda

In [4]:
from configs.credentials_config import API_KEY, ENDPOINT, MODEL, DEPLOYMENT # Crear archivo credentials_config.py con las credenciales de Azure OpenAI siguiendo el template

JSONS_FOLDER = "datasets/fallos_json"
OUTPUT_FOLDER = "datasets/fallos_vectorstore"

EMBEDDINGS_MODEL = "mixedbread-ai/mxbai-embed-large-v1"
CHUNK_SIZE = 1024
CHUNK_OVERLAP = 128

In [5]:
print("JSON files found:", glob.glob(os.path.join(JSONS_FOLDER, "**", "*.json"), recursive=True))

JSON files found: ['datasets/fallos_json/03/8985.json', 'datasets/fallos_json/03/8946.json', 'datasets/fallos_json/03/9029.json', 'datasets/fallos_json/03/9025.json', 'datasets/fallos_json/03/9024.json', 'datasets/fallos_json/03/9032.json', 'datasets/fallos_json/03/8947.json', 'datasets/fallos_json/03/8984.json', 'datasets/fallos_json/03/9019.json', 'datasets/fallos_json/03/9035.json', 'datasets/fallos_json/03/9023.json', 'datasets/fallos_json/03/8887.json', 'datasets/fallos_json/03/8940.json', 'datasets/fallos_json/03/8852.json', 'datasets/fallos_json/03/8916.json', 'datasets/fallos_json/03/8613.json', 'datasets/fallos_json/03/8903.json', 'datasets/fallos_json/03/8939.json', 'datasets/fallos_json/03/8997.json', 'datasets/fallos_json/03/8996.json', 'datasets/fallos_json/03/9020.json', 'datasets/fallos_json/03/8959.json', 'datasets/fallos_json/03/8840.json', 'datasets/fallos_json/03/9031.json', 'datasets/fallos_json/03/8973.json', 'datasets/fallos_json/03/8932.json', 'datasets/fallos_js

In [6]:
# Lee múltiples fallos en JSON
print("[INFO] Cargando y dividiendo los fallos JSON...")
splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

chunks = []

for path in glob.glob(os.path.join(JSONS_FOLDER, "**", "*.json"), recursive=True):
    with open(path, encoding="utf-8") as f:
        fallos = json.load(f)          # cada archivo es un array con 1 fallo

    for fallo in fallos:
        contenido = fallo.get("CONTENIDO", {})
        for seccion, parrafos in contenido.items():
            texto = "\n".join(parrafos)            # solo esa sección
            for frag in splitter.split_text(texto):
                chunks.append(
                    Document(
                        page_content = frag,
                        metadata = {
                            "seccion": seccion,
                            "fuente" : os.path.basename(path)
                        }
                    )
                )

print(f"[INFO] Total de fragmentos generados: {len(chunks)}")

# --- Embeddings y FAISS ---
print("[INFO] Generando embeddings...")
embedding_model = LangchainSentenceTransformer()
texts, embeddings = embedding_model.embed_documents(chunks)


[INFO] Cargando y dividiendo los fallos JSON...
[INFO] Total de fragmentos generados: 4930
[INFO] Generando embeddings...
[INFO] Filtering 4930 chunks...
[INFO] Embedding 4930 clean chunks...


Batches: 100%|██████████| 155/155 [12:52<00:00,  4.99s/it]


In [13]:
texts

['"URBANOTEC S.A. S/ QUIEBRA" - Expte. Nº 8985',
 'En la ciudad de Paraná, capital de la provincia de Entre Ríos, a los veinticinco días del mes de marzo del año dos mil veinticuatro reunidos los integrantes de este Tribunal asistidos por el Secretario autorizante, para conocer el recurso de inaplicabilidad de ley deducido en fecha 20/10/2023 en las actuaciones:\n"URBANOTEC S.A. S/ QUIEBRA" - Expte. Nº 8985, respecto de la resolución de la Sala Civil y Comercial de la Cámara de Apelaciones de Gualeguaychú dictada en fecha 19/10/2023. Que la votación debe tener lugar en el siguiente orden: Sra. Vocal Dra. Gisela N. Schumacher; Sr. Vocal Dr. Leonardo Portela y Sra. Vocal Dra. Laura Mariana Soage.\nEstudiados los autos, la Sala se planteó la siguiente cuestión: ¿qué corresponde resolver respecto del recurso de inaplicabilidad de ley interpuesto?\nA LA CUESTIÓN PROPUESTA LA SRA. VOCAL DRA.',
 '1.- La sentencia de Camara que viene recurrida La sentencia de la Cámara de Apelaciones Sala Prim

In [14]:
text_embedding_pairs = list(zip(texts, embeddings))
vectorstore = FAISS.from_embeddings(text_embedding_pairs, embedding_model)
vectorstore.save_local("datasets/fallos_vectorstore")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [15]:
def create_context(query):
    results = vectorstore.similarity_search(query, k=5)
    # for i, doc in enumerate(results, 1):
    #     print(f"{i}. {doc.page_content.strip()}\n")
    return results

azure_client = AsyncAzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=ENDPOINT,
    api_key=API_KEY
)

async def get_response(query):
    res = await azure_client.chat.completions.create(
        model=DEPLOYMENT,
        messages=[
            {
                "role": "system",
                "content": "Sos un asistente judicial especializado en encontrar semejanzas entre fallos y responder preguntas sobre ellos. Tu tarea es analizar el contexto y responder de manera precisa y concisa.",
            },
            {
                "role": "system", 
                "content": f"El contexto es: {create_context(query)}"
            },
            {
                "role": "user",
                "content": query
            }
        ],
        
    )
    return res

In [22]:
response = await get_response("Explicame el fallo que dice: TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS -  Expte. Nº 8142")

In [23]:
respuesta = response.choices[0].message.content
respuesta


'El fallo en el caso "TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS" - Expte. Nº 8142, se centra en una solicitud de regulación de honorarios relacionada con la actuación profesional en recursos de inaplicabilidad de ley. Los hechos se presentan ante el tribunal debido a la petición realizada el 7 de diciembre de 2023, que se basa en actuaciones llevadas a cabo anteriormente, específicamente en resoluciones de fecha 25 de junio de 2020 y 10 de abril de 2023.\n\nEl tribunal toma en consideración la normativa arancelaria correspondiente, que incluye varios artículos de la ley 7046, para determinar la regulación de honorarios de los profesionales involucrados en el caso. La base económica utilizada para esta liquidación se aprobó en una fecha anterior, el 8 de junio de 2023.\n\nEn resumen, el fallo se ocupa de la regulación de los honorarios de los abogados en el contexto de un proceso judicial que implica cobro de dinero, asegurando que se sigan la

In [24]:
from IPython.display import Markdown
display(Markdown(respuesta))


El fallo en el caso "TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS" - Expte. Nº 8142, se centra en una solicitud de regulación de honorarios relacionada con la actuación profesional en recursos de inaplicabilidad de ley. Los hechos se presentan ante el tribunal debido a la petición realizada el 7 de diciembre de 2023, que se basa en actuaciones llevadas a cabo anteriormente, específicamente en resoluciones de fecha 25 de junio de 2020 y 10 de abril de 2023.

El tribunal toma en consideración la normativa arancelaria correspondiente, que incluye varios artículos de la ley 7046, para determinar la regulación de honorarios de los profesionales involucrados en el caso. La base económica utilizada para esta liquidación se aprobó en una fecha anterior, el 8 de junio de 2023.

En resumen, el fallo se ocupa de la regulación de los honorarios de los abogados en el contexto de un proceso judicial que implica cobro de dinero, asegurando que se sigan las disposiciones legales pertinentes.